# **POKY XEASY Spectrum to UCSF Spectrum Converter**

by **Woonghee Lee, Ph.D.** (woonghee.lee@ucdenver.edu; POKY Team, Department of Chemistry, University of Colorado Denver)

June 20, 2023


Reference: https://github.com/yokochi47/xyza2pipe, https://nmrglue.readthedocs.io/en/latest/examples/pipe2sparky_2d.html


POKY download: https://poky.clas.ucdenver.edu

In [ ]:
#@title ### **Installing XYZA2PIPE and NMRGLUE**
!git clone --quiet https://github.com/yokochi47/xyza2pipe.git > /dev/null
!cd xyza2pipe; make -s
!pip install -q nmrglue

In [ ]:
#@title ### **Upload .16 and .param files. It must be two matching files.**
from google.colab import files
uploaded = files.upload()
uploaded_list = list(uploaded.keys())
if len(uploaded_list) != 2:
  print('Please uploaded matching two .16 and .param files.')
  raise SystemError
for u in uploaded_list:
  if u.endswith('.16'):
    fs = u
  elif u.endswith('.param'):
    fp = u

In [ ]:
#@title ### **Convert**
x2p = 'xyza2pipe/xeasy2pipe'

import os
fd = open(fp, 'r')
lines = fd.readlines()
fd.close()


o_list = [0, 0, 0, 0]
for line in lines:
  if line.find('Spectral sweep width in w') == 0:
    dim = int(line.split()[4][1])-1
    sw = float(line.split()[-1])
    o_list[dim] = sw

for line in lines:
  if line.find('Maximum chemical shift in w') == 0:
    dim = int(line.split()[4][1])-1
    mc = float(line.split()[-1])
    o_list[dim] = mc - o_list[dim]/2

prefix = fs.split('.')[0]

if o_list[3] != 0:
  c = f'--xCAR {o_list[0]} --yCAR {o_list[1]} --zCAR {o_list[2]} --aCAR {o_list[3]}'
elif o_list[2] != 0:
  c = f'--xCAR {o_list[0]} --yCAR {o_list[1]} --zCAR {o_list[2]}'
else:
  c = f'--xCAR {o_list[0]} --yCAR {o_list[1]}'

cmd = f'{x2p} {c} -i {fs} > {prefix}.pipe'
os.system(cmd)

import nmrglue as ng
dic, data = ng.pipe.read(f'{prefix}.pipe')
udic = ng.pipe.guess_udic(dic, data)
C = ng.convert.converter()
ndim = udic['ndim']

for i in range(ndim):
  if udic[i]['label'] == 'N':
    udic[i]['label'] = '15N'
  elif udic[i]['label'] == 'C':
    udic[i]['label'] = '13C'
  elif udic[i]['label'] == 'H':
    udic[i]['label'] = '1H'
  elif udic[i]['label'] == 'F':
    udic[i]['label'] = '19F'
  elif udic[i]['label'] == 'P':
    udic[i]['label'] = '31P'

C.from_pipe(dic, data, udic)

ng.sparky.write(f'{prefix}.ucsf', *C.to_sparky(), overwrite=True)

In [15]:
#@title ### **Download UCSF Spectrum**
files.download(f'{prefix}.ucsf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>